## FBREF SCRAPING TOOL

This file is to scrape tables from FBREF website and store it as a Pandas Dataframe. You can also save the dataframe as a csv or xlsx file. 

### Directions

You only need to change Cell 7 and then run all the cells. 

The instructions are given there.

You should see a sample of the scraped table before it is saved in a file wherever you have stored this notebook on your console.

In [1]:
# Convert html table to pandas dataframe 
import pandas as pd

# Webpage and Captcha handling
import requests
from selenium import webdriver
from seleniumbase import SB
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Getting data from webpage 
from bs4 import BeautifulSoup
import codecs
import re
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Fbref generally has multi-level columns which this custom join merges as one by hyphenating and appending them 
def custom_join(columns):
    col1, col2 = columns
    if col1.startswith("Unnamed") and col2.startswith("Unnamed"):
        return col1
    elif col1.startswith("Unnamed"):
        return col2
    elif col2.startswith("Unnamed"):
        return col1
    else:
        return f"{col1}-{col2}"

In [3]:
# Gets the html source code of the page. Some tables are also located in the comments so it uncomments
# the tables as well
def get_html_source(url, keyword="Premier League"):
    with SB(uc=True, test=True) as sb:
        sb.driver.uc_open_with_reconnect(url, 3)
        if not sb.is_text_visible(keyword):
            sb.driver.uc_open_with_reconnect(url, 4)
        sb.assert_text(keyword, timeout=3)
        html_source = sb.get_page_source().replace('<!--','').replace('-->','')
    return html_source


In [4]:
# Finds the table with the given table_id
def get_table_df(html_source, table_id):
    soup = BeautifulSoup(html_source, 'html.parser')
    table = soup.find('table', id=table_id)
    df = pd.read_html(str(table))[0]
    return df

In [11]:
# Cleans the table of null values and drops the given list of columns. 
# Also excludes players that have not played the "min_matches" number of games.
def clean_table(df, min_matches, cols_drop_list=['Rk', 'Nation', 'Born', 'Matches']):
    df.columns = df.columns.map(custom_join)
    df = df.dropna(subset=['Rk'])
    df = df[df["Player"] != "Player"]
    df = df.drop(cols_drop_list, axis=1)
    df = df[df["90s"].astype(float) >= min_matches]
    return df

In [14]:
# Saves the table in csv or xlsx format
# Default is csv
def save_df(filename, is_excel=False):
    if is_excel:
        df.to_excel(filename+".xlsx", index=False)
    else:
        df.to_csv(filename+".csv", index=False)

In [21]:
# Steps:
# 1. Add URL to scrape. Ensure that it contains the table you want.
# 2. Add keywords that adds a security check that the right page is being scraped
# 3. Look up the table id by accessing the html code of the page (command + option + I / ctrl + shift + I)
# 4. List the columns you want to drop from the table
# 5. Set the min number of matches played for a player to be included in the dataset

url = "https://fbref.com/en/comps/Big5/defense/players/Big-5-European-Leagues-Stats"
keyword = "Big 5 European"
table_id = "stats_defense"
cols_drop_list=['Rk', 'Nation', 'Born', 'Matches']
min_matches = 4 #(360 minutes ~ 1 min/game)


html_source = get_html_source(url, keyword)
df = get_table_df(html_source, table_id)
df = clean_table(df, min_matches, cols_drop_list)
df.head()
save_df("Big_5_23_24_Defensive_Actions")

==================== {  File "<frozen runpy>:198:SB} starts ====================
=============== {  File "<frozen runpy>:198:SB} passed in 16.08s ===============
